# DataFrame Methods More

In this chapter, we cover several more less common, but still useful and important DataFrames methods that you need to know in order to be fully capable at analyzing data with pandas. 

* `idxmax` and `idxmin` - Return the index of the max/min
* `agg` - Compute multiple aggregations at once
* `nsmallest`/`nlargest` - Retun the top/bottom `n` values
* `diff` and `pct_change` - Find the difference/percent change from one value to the next
* `corr` - Computes the correlation between each pair of numeric columns
* `sample` - Randomly sample values in a Series
* `replace` - Replace one or more values in a variety of ways

Let's read in the movie dataset with the title in the index and select just the numeric columns.

In [ ]:
import pandas as pd
movie = pd.read_csv('../data/movie.csv', index_col='title').select_dtypes('number')
movie.head()

## The `idxmax` and `idxmin` methods

The `idxmax` and `idxmin` methods return the index where the maximum value occurs for each column. When we call the `idxmax` method on our DataFrame, we learn that the movie with longest duration is 'Trapped', the movie with the highest gross is 'Avatar', the one highest IMDB score is 'Towering Inferno', etc...

In [ ]:
movie.idxmax()

## The `agg` method

The `agg` method allows us to calculate several aggregations at once by providing it a list of the aggregation methods as strings. Here, we find the min, max, and index for the minimum and maximum for each column

In [ ]:
aggs = movie.agg(['min', 'idxmin', 'max', 'idxmax', 'nunique'])
aggs

This returned data might be easier to read when transposed. Let's transpose the results with the `T` attribute.

In [ ]:
aggs.T

## The `nlargest/nsmallest` methods
The `nlargest` and `nsmallest` methods provide a similar solution that `sort_values` does. Pass them `n`, the number of rows you want to return and `columns`, a string of a column name you would like to use to determine the ordering.  The following returns the movies with the three highest gross values.

In [ ]:
movie.nlargest(3, 'gross')

It is possible to duplicate this with `sort_values` together with the `head` method.

In [ ]:
movie.sort_values('gross', ascending=False).head(3)

### Why use `nlargest/nsmallest`?
While `nlargest/nsmallest` can be duplicated with `sort_values`, in theory, `nlargest/nsmallest` should perform better as it uses a [selection algorithm][1] and not a sorting one. The `nlargest/nsmallest` methods also have the ability to keep the top n rows with ties with the `keep` parameter. 

[1]: https://en.wikipedia.org/wiki/Selection_algorithm

## The `diff` and `pct_change` methods

The `diff` and `pct_change` methods work just as they do on a Series. Let's read in the `stocks10` dataset which contains the closing stock price for ten stocks beginning from 2010.

In [ ]:
stocks = pd.read_csv('../data/stocks/stocks10.csv', index_col='date', parse_dates=['date'])
stocks.head()

The `diff` method takes the difference between the current value and the nth value preceding it. Below, we get the change in price from two trading days prior.

In [ ]:
stocks.head().diff(2)

The `pct_change` return the percentage change as a fraction. Here, we round the number and multiply by 100 so the results are easier to interpret.

In [ ]:
stocks.head().pct_change(2).round(3) * 100

## The `corr` method

The `corr` method computes the correlation between every pair of numeric columns in the DataFrame. The resulting DataFrame is 'square' meaning it has the same number of rows as it does columns. The row names and column names are the same. Also, the DataFrame is 'symmetric' meaning that taking the transpose results in the exact same DataFrame. Here, we find the correlation between each stocks. Notice that the technology stocks, MSFT, AAPL, AMZN, and FB are highly correlated to one another. The energy stocks XOM and SLB are highly correlated with one another and less correlated with the technology stocks.

In [ ]:
stocks.corr().round(2)

Series also have a `corr` method. You must pass it a Series to find its correlation.

In [ ]:
stocks['MSFT'].corr(stocks['AAPL'])

## The `sample` method

The `sample` method randomly samples rows or columns from the DataFrame. Here, we select three random rows. By default, sampling is done without replacement, so these will be three rows.

In [ ]:
movie.sample(3)

It's possible to randomly sample columns by setting the `axis` parameter to 'columns' or 1.

In [ ]:
movie.sample(5, axis='columns').head()

Use the `frac` parameter to select a random fraction of the rows and set `replace` equal to `True` to sample with replacement. Here, we select a random 25% of the rows with replacement.

In [ ]:
movie.sample(frac=.25, replace=True).shape

## The `replace` method

The `replace` method can be used to replace values in your DataFrame. It is very powerful and flexible. It is also quite complex as there are many different combinations of parameters to handle a variety of different kinds of replacement. Let's read in the first 5 rows of the San Francisco employee compensation dataset dropping the year column. Each numeric column is rounded to the nearest ten-thousand.

In [ ]:
sf_emp_head = pd.read_csv('../data/sf_employee_compensation.csv', nrows=5)
sf_emp_head = sf_emp_head.drop(columns='year').round(-4)
sf_emp_head

The `replace` method has two main parameters, `to_replace` and `value`. The simplest application is to set each one to a single value. Below, we replace all of the values equal to 10,000 with 9,999. All values in the entire DataFrame are searched to be replaced.

In [ ]:
sf_emp_head.replace(to_replace=10000, value=9999)

The `replace` method can also replace exact strings.

In [ ]:
sf_emp_head.replace(to_replace='Public Protection', value='PP')

Instead of using two parameters, you can set `to_replace` to a dictionary to map the old value to the new value. When using a dictionary, you do not use the parameter `value`. Below, we replace 'Community Health' with 'Health'.

In [ ]:
sf_emp_head.replace(to_replace={'Community Health':'Health'})

You can replace as many values as you'd like with a dictionary. The first parameter is `to_replace`, so we can call this method without explicitly providing the parameter name. We import `numpy` to help replace all zeros with missing values.

In [ ]:
import numpy as np
sf_emp_head.replace({'Community Health':'Health', 0: np.nan})

### Specifying which columns to search for replacement

Calling `replace` as we did above, replaces all values in all columns that match the value to replace. Instead, we might be interested in only replacing values in a particular column, or replacing the same value with different values depending on the column.

We can specify which columns to replace which values by passing in a dictionary of dictionaries, where the keys of the dictionary specify the column names and the values are the dictionary of original values mapped to their replacement. Take a look at the following dictionary. When passed to the `replace` method, it instructs it to replace 0 with nan and 60000 with 99999 for just the overtime column. The retirement column will have 0 replaced with -999.

```python
{'overtime':{0: np.nan, 
             60000: 99999}, 
 'retirement': {0: -999}}
```

Let's use this dictionary to make the specific replacement.

In [ ]:
sf_emp_head.replace({'overtime':{0: np.nan, 60000:99999}, 'retirement': {0:-999}})

### Replacing Substrings

As we've seen before, the `replace` method searches for exact strings. Attempting to replace 'Public' with 'Pub.' will do nothing in our DataFrame as there is no exact value 'Public'.

In [ ]:
sf_emp_head.replace({'Public':'Pub.'})

This does nothing because pandas is looking for the exact value 'Department'. The word 'Department' does appear in many strings but it is not the only word in the string. Let's run the same command but set `regex` to `True`.

In [ ]:
sf_emp_head.replace({'Department':'dept'}, regex=True)

## Methods Available only to Series and not DataFrames
There are more than a few methods that are available only to Series objects, but the following are the most important.

### No `str` or `dt` accessor
DataFrames have no special methods just for strings or datetimes. There is no `str` or `dt` accessor. They can only be used on Series objects.

### No `unique` or `value_counts`
Both `unique` and `value_counts` are only available to Series as well.

## Exercises

Execute the following cell to read in the City of Houston dataset and use it to answer the next exercise.

In [ ]:
emp = pd.read_csv('../data/employee.csv')
emp.head(3)

### Exercise 1

<span  style="color:green; font-size:16px">Find the relative frequency of departments for all employees and then find the relative frequency of departments for the top 100 salaries. Compare the differences.</span>

### Exercise 2

<span  style="color:green; font-size:16px">Find the day that each stock had its largest percentage one-day drop in price.</span>